# Analzying Driver Location Data

The objective of this program is to better help the company's managers understand where the company's Sales Representatives (each with their own respective company vehicle) are driving. Each car is fit with a GPS Tracker that provides a central database with the Latitude & Longitude coordinates every minute.

Since the current visualization tools are inadequate (to say the least ;), this tool aims to improve the general understanding of where the company is represented and how it may improve hereon.

~Last update: 12/10/2022

### Install general dependencies & import all files in the directory 

In [1]:
import numpy as np
import pandas as pd

files = []
driver = []
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        file = os.path.join(dirname, filename)
        files.append(file)
        print(os.path.join(dirname, filename))
        


/kaggle/input/sale-reps/TIMOTHY Report.csv
/kaggle/input/sale-reps/KAMALUDEEN Report.csv
/kaggle/input/sale-reps/MUSA Report.csv
/kaggle/input/sale-reps/Adeosun Report.csv
/kaggle/input/sale-reps/CHRISTIAN Report.csv
/kaggle/input/sale-reps/DANIEL Report.csv
/kaggle/input/sale-reps/JEJELOLA Report.csv
/kaggle/input/sale-reps/PETER Report.csv
/kaggle/input/sale-reps/MUDASSIR Report.csv
/kaggle/input/sale-reps/AKINKUMI Report.csv
/kaggle/input/sale-reps/HILLARY Report.csv


### Create a script that cleans and removes duplicate columns

- Drop unneccesary columns
- Rename Columns with the right labels
- Remove duplicate & wrong columns

In [2]:
drivers = []

def clean_df(p):
    driver = p.iloc[0,0].replace("Device: ", "")
    p.drop(0, axis = 0, inplace = True)
    p.drop('Time', axis = 1, inplace = True)
    p.rename(columns = {'Unnamed: 1': 'Time'}, inplace = True)
    null_values = p[(p['lat']== 0) | (p['lng'] == 0)].index      # Checks whether 'lat' or 'lng' = 0.0
    p.drop_duplicates(inplace = True)
    p.drop(null_values, axis = 0, inplace = True)
    p['driver'] = driver.strip()
    drivers.append(driver)
    return p

### Import the csv files and parse the data using the above create cleaning function

In [3]:
%%time
# Read in and clean the files

all_reps = []

i = 0

for file in files:
    df = pd.read_csv(file, usecols = ['Time','Unnamed: 1','lat', 'lng'], 
                     dtype = {'Time': 'object', 'lat': 'float32', 'lng': 'float32'})
    df = clean_df(df)
    all_reps.append(df)
    
result = pd.concat(all_reps)
result['driver'] =result['driver'].astype('category') 
result.drop_duplicates(subset = ['lat', 'lng', 'driver'],keep = 'first', inplace = True)

result['Time'] = pd.to_datetime(result['Time'],format = '%-m/%-d/%Y %-I:%M:%S %p', infer_datetime_format=True)
result.set_index('Time', inplace = True)

print(result['driver'].value_counts())

print('\n', result.shape)
print('\n', result.info())
result.head()

CHRISTIAN OGBODE         17727
MUDASSIR DANLADI         16069
MUSA ISHAYA              15788
HILLARY ARIWA            15154
TIMOTHY ADELANI          13224
AKINKUMI IDOWU           13208
PETER YOHANA             12817
ADEOSUN OLUWASIJI        11951
KAMALUDEEN ABDULWAHAB     6696
JEJELOLA OMOTAYO          1566
DANIEL OJONYE             1139
Name: driver, dtype: int64

 (125339, 3)
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 125339 entries, 2023-01-29 00:00:05 to 2023-02-28 17:20:30
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype   
---  ------  --------------   -----   
 0   lat     125339 non-null  float32 
 1   lng     125339 non-null  float32 
 2   driver  125339 non-null  category
dtypes: category(1), float32(2)
memory usage: 2.0 MB

 None
CPU times: user 17.8 s, sys: 558 ms, total: 18.4 s
Wall time: 19.6 s


,lat,lng,driver
Time,,,
2023-01-29 00:00:05,7.49346,3.96049,TIMOTHY ADELANI
2023-01-29 01:30:14,7.49344,3.96046,TIMOTHY ADELANI
2023-01-29 01:33:31,7.49347,3.96052,TIMOTHY ADELANI
2023-01-29 01:47:10,7.49348,3.96047,TIMOTHY ADELANI
2023-01-29 09:33:45,7.49358,3.96051,TIMOTHY ADELANI


### Use Plotly Express & Mapbox to visualize the driver locations on an interactive map

This setup requires an account & API Token to use Mapbox

In this case, the drivers are still manually configured. In the ideal situation however, this will be derived from the csv file directly.... Currently still something to work on.

In [4]:
[i for i in result.columns]

['lat', 'lng', 'driver']

In [5]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
import plotly.express as px

token = user_secrets.get_secret("mapbox_token")

px.set_mapbox_access_token(token)

fig = px.scatter_mapbox(result, lat= 'lat', lon= 'lng', 
                        zoom = 4,title = 'Sales Reps Trip Summary February 2023', 
                        color = 'driver')


In [6]:
# Export the generated report to an html file
fig.write_html("./February Trip Report.html", include_plotlyjs="cdn")
